# Training: SQL (Heavy-users) 💪
Welcome to the training notebook on using SQL.


This notebook is pitched at heavy users who perform data management rples to store objects permanently in SQL.

![SQL Futurama meme](https://live.staticflickr.com/65535/49188193126_3b991de60e_z.jpg "SQL Futurama meme") 

# What will this session cover? 👂
This session will show you how to do the following things in SQL:
1. Dynmaic SQL querying
1. Informal guide to when you should permanently store SQL objects
1. Creating and updating tables
1. Importing data into SQL
1. Indexing columns to improve querying speeds
1. Adding constraints to columns to restrict entries that can go inside it
1. Using stored procedures and functions to do more bespoke operations
1. Database triggers
1. Version-controlling databases

In [4]:
-- set database to use
USE [HEFE-AN-DEV];

# 1. Dynamic SQL querying
This is essentially SQL code that writes SQL code. It's particularly useful when you want to parameterise your SQL statements with an input variable. 

To be able to employ dynamic SQL querying, you'll need to know how to:
1. Create a variable in SQL and assign it a value
1. Create a SQL query which takes the variable you created
1. Execute the SQL query

The core idea behind dynamic SQL querying lies in being able to write your query as a string/varchar/text, and then execute/run the text as if it is a SQL query.

> **USER STORY:** *As a lazy-ass, punk-ass, funk-ass SQL version of NAS, I want to quickly run a SQL statement to quickly count all the number of rows in several tables in my database rather than write the same* `SELECT COUNT(*) FROM <*table_name*>` *statement several times but with different table names, so I can get back to making my new coding mixtape.*



In [9]:
-- create and set variable/parameters
DECLARE @name_schema AS NVARCHAR(25) = 'Production'
DECLARE @name_table AS NVARCHAR(50) = 'Product'

-- write dynamic SQL as string/text
DECLARE @query AS NVARCHAR(MAX) = 
'
    SELECT COUNT(*)
    FROM [' + @name_schema + '].[' + @name_table + '];
' 

-- print query to see what SQL is written
PRINT @query

-- excute dynamic SQL being written as string/text
EXEC sp_executesql @query

## EXERCISE: Dynamic SQL querying #1
**Question:** Is there an even more efficient way to perform the same task in (1.) without having to use dynamic SQL querying?

**Hint:** Consider using System Tables belonging to the `[INFORMATION_SCHEMA]`.

**Note:** This exercise introduces you to some very useful tables that exist in the background which will be extremely useful for data management tasks. It also shows that dynamic SQL is not always the best option.

In [0]:
-- please write your answer here

## EXERCISE: Dynamic SQL querying #2
**Question:** Recall in the *Training: SQL (Medium-user)* notebook the instructions for how you pivot tables in SQL. Namely, how you have to explicitly know and code the rows in the column you want to pivot to. This can be tedious, especially when you have many unique rows to pivot to. There is a way to pivot without explicitly specifying the rows you want to pivot to columns. Can you write a query that does this?

**Note:** This exercise demonstrates the power of dynamic querying.

# 2. Informal guide to what you should permanently store in SQL
The attitude to storing data in SQL should be different to storing data in a folder. As a database administrator/architect/manager, your role is to effectively govern the storage and access of data so that it can be intuitively found, easily accessed and fluidly recycled/reused.

This means that your data in SQL should meet the following criteria:
- [ ] Be persistent - it should be able to hold different versions of your data across time series. *e.g. HESA return the 2015/16 data two times in the year, so both versions should go in a singled table with an column idenitifyin what version youre using.*
- [ ] Folllow a style-guide - all naming must be consistent with each other so that users of the databse can navigatee easily to where they want to go.
- [ ] Open access - enable a wide range of people to access your data.



# 3. Creating and updating tables
When deciding that **temporary tables** are unsuitable for your usage because you want to increase access to the data explicitly for more people or people will be building off your query to view the **CTE**, then creating and updating a table to store permanently in a SQL database is appropriate. 

When creating a table, you may want to store it in **tidy data format** (covered in *Training: SQL (Medium-users))*) so the same data but in a different version or time can be imported to this, meaning you are **updating** the ewly-created table.

To create a table, you need to know a few things upfront such as the columns it will contain, their data types and version. 



``` SQL
-- template: create table query
CREATE TABLE [<schema_name>].[<table_name>] (
    [Column_1] AS <datatype>
    ,[Column_2] AS <datatype>
    ,...
);

-- template: update table 
UPDATE [<schema_name>].[<table_name>]
SET [column_n] = <new_value>
WHERE [column_j] = <filter_value>;
```

# 4. Importing data into SQL
After deciding that you want to store the data in a table on SQL, and having creatied the tables, you next job is to actually import the data into the table. There are several methods.
1. **SQL Server Import Wizard:** Good for relatively quick, one-time imports of datasets. Bad for multiple table imports of the same datasets.
1. `INSERT INTO [<schema_name>].[<table_name>] VALUES (row_1_value_1, row_1_value_2, ...), ...`: Good for realtively quick, one time imports of rows. Bad for anything outside of this.
1. **SSIS Package:** Good for efficiently importing multiple tables of the same dataset. Bad for people not wanting to learn a new software.
1. **R:** Good as most people know about how to do programme in this. Bad if you are trying to import a large amount of data.

We define *large* in point (4.) with regards to the size of the ***data being imported into SQL being larger than the amount of RAM your computer has.*** This is because importing data into SQL from R requires importing the data into your R session first, then moving it across into SQL. As R stores data on the computers RAM for fast retrieval (relative to storage on a hard disk), then if the data was larger than the RAM, this would be a very slow process. 

# 5. Indexing columns to improve querying speeds
With really big tables, it makes sense to **index** their columns so that it is much more faster to query from them via filtering/joining or other SQL operations.

**Column indexes/indices** are essentially a way of creating "bookmarks" in your data so that when you're filtering from it for instance, the SQL engine looks through the bookmarks and filters on them, rather than go into each entry, row-by-row, and filtering.

They do slow down `UPDATE` and `INSERT` operations so you may want to drop **indexes** before performing these operations and then reapply the **indexes**.

There are two types of **indexes**:
- **Clustered:** Physically orders the data on the ~~hard~~ disk. This can make `ORDER BY` operations significantly faster.
    + Only one can be created per table.
    + Faster to read than non-clustered index as data is physically stored in an index order.
    + Does not take additional memory to store.
- **Non-clustered:** Defines a "logical" order that does not match the physical order on the ~~hard~~ disk. This "logical" order is like a layer on top of a physical order, where a pointer is used.
    + More than one can be created per table as they can be applied to columns.
    + Data insertion/update is faster than clustered index. 
    + Needs additional memory to store.

![Clustered image](https://i.stack.imgur.com/kFSWR.png "Clustered image")

> **TIP:** Typically, **clustered indexes** are created on the most unique column or columns in your data, such as a unique identifier (also known in SQL parlance as the **primary key**).
>> For the most part, we use **non-clustered indexes** on columns which we plan to filter by and join on and drop them before inserting or updating data in the tables.

```
-- template: create index
CREATE NONCLUSTERED INDEX IX_<table_name>_<column_name>
    ON [<schema_name>].[<table_name>] ([<column_name>])
    -- enable faster index creation/rebuild times
    -- without this, can dramaticall increase tempdb file sizes, which is bad!
    WITH (SORT_IN_TEMPDB = ON);
```

```
-- template: drop multiple indexes, checking if they exist first
IF EXISTS 
(
    SELECT [name] FROM [sys].[indexes]
    WHERE [name] IN (N'IX_<table_name>_<column_name_1>', 'IX_<table_name>_<column_name_2>', ...)
) 
BEGIN
    DROP INDEX IX_<table_name>_<column_name_1>
        ON [<schema_name>].[<table_name>]
    DROP INDEX IX_<table_name>_<column_name_2>
         ON [<schema_name>].[<table_name>]
    ...
END

-- need to specify what to do if these indexes don't exist
-- otherwise SSIS package will throw error because won't know
-- what to do if indexes don't exist
ELSE 
BEGIN
    WAITFOR DELAY '00:00:00'
END;
```

# 6. Adding constraints to columns
Adding **constraints** to columns in you data ensures the integrity of possible values being entered into the table. They specify rules that data must adhere to in your tables.

Specifically of interest is the **check constraint** which sets a specified list of values that the column can take, and any data being entered into the table that does not fulfill this pre-specified list of values cannot be imported until this is resolved.

```
-- template: create constraint on existing table
ALTER TABLE [<schema_name>].[<table_name>]
ADD CONSTRAINT CHK_<constraint_name> CHECK ([<column_integer>] > n AND [<column_string>] IN ('string_1', 'string_2', ...))
```

```
-- template: drop constrain on existing table
ALTER TABLE [<schema_name>].[<table_name>]
DROP CHECK CHK_<constraint_name>
```

# 7. Using stored procedures and functions
**Stored procedures** and **functions** both execute a set of SQL instructions to return a result. Howeverm they are different in subtle ways.


Stored Procedures | Functions
--- | --- |
Does not return a value, just `0` or `n` values | Always returns a value
Can have input **and** output parameters | Only have input parameters for it
Cannot call them from **Functions** | Can call them from **Stored Procedures**


In [0]:
-- create a simple stored proc
CREATE PROCEDURE spr_HelloWorld
AS
PRINT 'Hello World'

-- execute stored proc
EXEC spr_HelloWorld

In [0]:
-- create a simple function
CREATE FUNCTION dbo.fn_helloworld()
RETURNS varchar(20)
AS 
BEGIN
	 RETURN 'Hello world'
END;

GO;

-- call the function
SELECT dbo.fn_helloworld()

# 8. Database triggers to record activity

In [0]:
-- create schema to assign database activity table in
CREATE SCHEMA [AdminDetails] AUTHORIZATION [dbo];
GO;

-- create table to record database activity
CREATE TABLE [AuditDetails].[DatabaseChangeLog](
	[ChangeId] [int] IDENTITY(1,1) NOT NULL
	,[ChangeDate] [datetime] NOT NULL 
		CONSTRAINT [DF_ddl_log_ChangeDate]
			DEFAULT (GETDATE())
	,[NameUser] [nvarchar](50) NOT NULL
		CONSTRAINT [DF_ddl_log_NameUser]   
            DEFAULT (CONVERT([nvarchar](50), USER_NAME(), (0)))
	,[NameSecurity] [nvarchar](50) NOT NULL
		CONSTRAINT [DF_DDLChangeLog_NameSecurity]   
            DEFAULT (CONVERT([nvarchar](50), SUSER_SNAME(), (0)))
	,[NameLogin] [nvarchar](50) NOT NULL
		CONSTRAINT [DF_DDLChangeLog_NameLogin]   
            DEFAULT (CONVERT([nvarchar](50), original_login(),(0)))
	,[EventType] [nvarchar](100) NULL
	,[ObjectName] [nvarchar](100) NULL
	,[ObjectType] [nvarchar](100) NULL
	,[TsqlCode] [nvarchar](max) NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY];
GO;

-- create and turn on database trigger
CREATE TRIGGER trg_DatabaseChangeLog ON DATABASE 
    FOR DDL_DATABASE_LEVEL_EVENTS 
AS 
    DECLARE @data XML 
    SET @data = EVENTDATA() 
    IF @data.value('(/EVENT_INSTANCE/EventType)[1]', 'nvarchar(100)') <> 'CREATE_STATISTICS'  
		INSERT  INTO [DatabaseChangeLog] 
		( 
			[EventType], 
			[ObjectName], 
			[ObjectType], 
			[TsqlCode]
		) 
		VALUES  
		( 
			@data.value('(/EVENT_INSTANCE/EventType)[1]', 'nvarchar(100)'), 
			@data.value('(/EVENT_INSTANCE/ObjectName)[1]', 'nvarchar(100)'), 
			@data.value('(/EVENT_INSTANCE/ObjectType)[1]', 'nvarchar(100)'), 
			@data.value('(/EVENT_INSTANCE/TSQLCommand)[1]', 'nvarchar(max)') 
		); 
GO

ENABLE TRIGGER [trg_DatabaseChangeLog] ON DATABASE;
GO;

-- create View to pull in schema name so can better identify tables
CREATE VIEW [AuditDetails].[vw_DatabaseChangeLog]
AS 
(
    SELECT changelog.[ChangeId]
        ,changelog.[ChangeDate]
        ,changelog.[NameUser]
        ,changelog.[NameSecurity]
        ,changelog.[NameLogin]
        ,changelog.[EventType]
        ,[SchemaId] = admintable.[schema_id]
        ,[SchemaName] = OBJECT_SCHEMA_NAME(admintable.[object_id])
        ,[ObjectId] = admintable.[object_id]
        ,changelog.[ObjectName]
        ,[ObjectType] = admintable.[type_desc]
        ,changelog.[TsqlCode]
    FROM [AuditDetails].[DatabaseChangeLog] AS changelog
    LEFT JOIN [sys].[all_objects] AS admintable
        ON changelog.[objectName] = admintable.[name]
    -- Keep only 'genuine'/real-people SQL users
    WHERE changelog.[NameUser] != 'dbo'
)

# 9. Version-controlling your database

Additional guidance is available on Github [here](https://github.com/avisionh/SQL-Titbits/wiki/User-Guide:-SQL-x-Git-Version-Control).